In [ ]:
%matplotlib inline

import seaborn as sns
import pandas as pd
import pysal as ps
from pysal.lib import weights
from pysal.explore import esda
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
zona_diss=gpd.read_file('datosZonas_Stgo_C2017.gpkg')

In [ ]:
f, ax = plt.subplots(1, figsize=(12,12))

zona_diss.plot(ax=ax, column='INMIGRANTE',cmap='jet', linewidth=0.4)
zona_diss.boundary.plot(ax=ax, color='black',linewidth=0.4,label='Zonas')

In [ ]:
print(zona_diss.shape,zona_diss.head())

In [ ]:
w_queen = weights.Queen.from_dataframe(zona_diss,idVariable='NZONA')
w_rook = weights.Rook.from_dataframe(zona_diss,idVariable='NZONA')
w_knn = weights.KNN.from_dataframe(zona_diss,k=3,ids='NZONA')
w_dist_1km = weights.DistanceBand.from_dataframe(zona_diss, 1000)

In [ ]:
f, ax= plt.subplots(1,sharey=True)
f.set_size_inches(12,6)

W={'Queen':w_queen,'Rook':w_rook,'KNN':w_knn,'Dist':w_dist_1km}
for kw in W.keys():
    w=W[kw]
    print(kw,'==> Cardinalities, neighbors, weights:',w.cardinalities[100],w.neighbors[100],w.weights[100])
    card=pd.Series(w.cardinalities)
    ax.hist(card, bins=15,histtype='step',label=kw)
    
ax.legend()

In [ ]:
f,axes = plt.subplots(2,2,sharey=True)
f.set_size_inches(14,14)
axs=[axes[0][0],axes[0][1],axes[1][0],axes[1][1]]

focus = zona_diss.loc[[100], ['geometry']]

for ax in axs:
    zona_diss.plot(ax=ax, facecolor='gray', linewidth=0.1)
    focus.plot(facecolor='red', alpha=1, linewidth=0, ax=ax)
    ax.set_xlim(350000,354000)
    ax.set_ylim(6283000,6286000)
    
for kw,ax in zip(W.keys(),axs):
    w=W[kw]
    ax.set_title("Vecindad %s de Zona 100"%kw)
    neis= zona_diss.loc[w[100], :]
    neis.plot(ax=ax, facecolor='lime', linewidth=0)
    buf = focus.centroid.buffer(1000)
    buf.plot(edgecolor='red', facecolor='none', ax=ax)
    zona_diss.centroid.plot(ax=ax, color='k',markersize=2)
    

In [ ]:
W={'Queen':w_queen,'Rook':w_rook,'KNN':w_knn,'Dist':w_dist_1km}

field='HOMBRES'
for kw,ax in zip(W.keys(),axs):
    # Row-standardize the queen matrix
    w=W[kw]
    w.transform = 'R'
    mean=np.mean(zona_diss[field])
    std=np.std(zona_diss[field])
    normed=(zona_diss[field]-mean)/std
    zona_diss['%s_norm'%field]=normed
    # Compute spatial lag of `imd_score`
    w_lag = pd.Series(weights.lag_spatial(w, normed),index=normed.index)
    zona_diss['%s_%s'%(kw,field)] = w_lag
   
print(zona_diss.head())


In [ ]:
f,axes = plt.subplots(2,2,sharey=True)
f.set_size_inches(14,14)
axs=[axes[0][0],axes[0][1],axes[1][0],axes[1][1]]

for kw,ax in zip(W.keys(),axs):
    w=W[kw]
    x=zona_diss['%s_norm'%field]
    y=zona_diss['%s_%s'%(kw,field)]
    m,b= np.polyfit(x, y, 1)
    ax.plot(x,y,'.')
    ax.plot(x,m*x+b,ls='--')
    ax.set_xlabel(field)
    ax.set_ylabel('Spatial Lag - %s' %kw)
    ax.set_xlim(-1,3)
    ax.set_ylim(-1,3)
    ax.set_title(kw)
    ax.axhline(y=0,ls='--')
    ax.axvline(x=0,ls='--')

    mi = esda.Moran(x, w)
    ax.text(1,-0.5,'I Moran:%.2f, p_sim:%.4f'%(mi.I,mi.p_sim))


In [ ]:
import matplotlib.patches as mpatches
w=W['Queen']
x=zona_diss['%s_norm'%field]
lisa = esda.Moran_Local(x, w)
print(lisa.q)
zona_diss['quadrant'] = lisa.q
zona_diss['significant'] = lisa.p_sim < 0.05

quads={1:'HH',2:'LH',3:'LL',4:'HL'}

palette = {'LL': 'blue','HH':'red','LH':'#83cef4','HL':'#e59696'}
               
zona_diss['quad_class'] = [quads[x] for x in zona_diss['quadrant']]

f, ax = plt.subplots(1, figsize=(9, 9))

sig=zona_diss[zona_diss['significant']]
zona_diss.plot(color='gray',ax=ax)

handles=[]
for ctype, data in sig.groupby('quad_class'):
    
    # Define the color for each group using the dictionary
    color = palette[ctype]
    
    # Plot each group using the color defined above
    hand=data.plot(color=color,ax=ax,label=ctype)
    handles.append(mpatches.Patch(color=color, label=ctype))

plt.legend(handles=handles,loc='best')
ax.set_axis_off()
f.suptitle('LISA para %s'%field, size=12)


